#2024 51MCM-B题#

首先创建路径邻接矩阵，1代表联通，0代表断开。节点与自身不连通。
然后使用广度优先搜索，搜索出符合条件的五条路径

In [1]:
import numpy as np

matrix = np.array([[0, 1, 0, 1, 0, 0, 0, 0, 0],[1, 0, 1, 0, 1, 0, 0, 0, 0],[0, 1, 0, 0, 0, 1, 0, 0, 0],[1, 0, 0, 0, 1, 0, 1, 0, 0],[0, 1, 0, 1, 0, 1, 0, 1, 0],[0, 0, 1, 0, 1, 0, 0, 0, 1],[0, 0, 0, 1, 0, 0, 0, 1, 0],[0, 0, 0, 0, 1, 0, 1, 0, 1],[0, 0, 0, 0, 0, 1, 0, 1, 0]])
#print(matrix)

from collections import deque

class Graph:
    def __init__(self):
        self.graph = {}

    def add_edge(self, node, neighbors):
        self.graph[node] = neighbors

class ResizableList(object):
    def __init__(self, initial_size=90):
        self.data = [0] * initial_size
        self.size = 0
 
    def __getitem__(self, index):
        if index >= self.size:
            self._resize(index + 1)
        return self.data[index]
 
    def __setitem__(self, index, value):
        if index >= self.size:
            self._resize(index + 1)
        self.data[index] = value
        if index >= self.size:
            self.size = index + 1
 
    def _resize(self, new_size):
        self.data += [0] * (new_size - len(self.data))

In [2]:

g = Graph()
g.add_edge(1, [2,4])
g.add_edge(2, [1,3,5])
g.add_edge(3, [2,6])
g.add_edge(4, [1,5,7])
g.add_edge(5, [2,4,6,8])
g.add_edge(6, [3,5,9])
g.add_edge(7, [4,8])
g.add_edge(8, [5,7,9])
g.add_edge(9, [6,8])

def bfs(graph, start, target):
    visited = []
    paths = []
    queue = [[start]]
    if start == target:
        return "Start = target"
    while queue:
        path = queue.pop(0)
        node = path[-1]
        neighbours = graph[node]
        for neighbour in neighbours:
            new_path = list(path)
            new_path.append(neighbour)
            queue.append(new_path)
            if neighbour == target:
                if len(paths) >= 1:
                    if len(new_path) > len(paths[-1]):
                        return paths
                paths.append(new_path)
        visited.append(node)
    return "No path"

print(g.graph)
paths = bfs(g.graph, 3, 7)
print(paths)

{1: [2, 4], 2: [1, 3, 5], 3: [2, 6], 4: [1, 5, 7], 5: [2, 4, 6, 8], 6: [3, 5, 9], 7: [4, 8], 8: [5, 7, 9], 9: [6, 8]}
[[3, 2, 1, 4, 7], [3, 2, 5, 4, 7], [3, 2, 5, 8, 7], [3, 6, 5, 4, 7], [3, 6, 5, 8, 7], [3, 6, 9, 8, 7]]


##下面首先平均分配各个路径的流量，随后通过……我也不知道怎么做去完成它……

In [3]:
plans = [[1,5,100],
[1,6,150],
[1,8,200],
[1,9,180],
[2,4,160],
[2,6,110],
[2,7,120],
[2,9,140],
[3,4,130],
[3,5,150],
[3,7,140],
[3,8,160],
[4,2,100],
[4,3,170],
[4,8,190],
[4,9,120],
[5,1,160],
[5,3,140],
[5,7,180],
[5,9,100],
[6,1,170],
[6,2,120],
[6,7,130],
[6,8,200],
[7,2,120],
[7,3,150],
[7,5,190],
[7,6,100],
[8,1,130],
[8,3,180],
[8,4,140],
[8,6,150],
[9,1,170],
[9,2,130],
[9,4,190],
[9,5,160]]

def routenum(i,j):
    return 9 * i + j

def relationmat(route):
    relmat = ResizableList()
    while route:
        if len(route) == 1:
            break
        i = route.pop(0)
        j = route[0]
        relmat[routenum(i,j)] = 1
    return relmat

relmat = []
for plan in plans:
    i = plan[0]
    j = plan[1]
    routes = bfs(g.graph, i, j)
    plan.append(len(routes))
    for route in routes:
        relmat.append(relationmat(route).data)

nprelmat = np.array(relmat)
nprelmat = nprelmat.T
print(nprelmat)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]


##下面使用目标函数min=\Sigmax^2进行梯度下降优化

In [31]:
Q = nprelmat.T @ nprelmat
print(Q)

A = []
b = []
i = 0
for plan in plans:
    A.append(np.zeros(i).tolist() + np.ones(plan[3]).tolist() + np.zeros(len(Q) - plan[3] - i).tolist())
    b.append(plan[2])
    i += plan[3]

A = np.array(A)
print(A) 

#此处论文中的方法不可用，因为原来这个矩阵就已经是最简行阶梯了，前面的N列不是满秩的，所以无法求逆
P = np.append(np.append(Q, -A.T, axis=1), np.append(A, np.zeros((len(A), len(A))), axis=1), axis=0)
print(P)

np.linalg.matrix_rank(P)
#res = np.linalg.inv(P) @ np.append(np.zeros(len(Q)), b)
#print(res)
res=np.linalg.pinv(P)@np.append(np.zeros(len(Q)), b)
res[:len(Q)]

[[2 0 1 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [1 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 1]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 1 0 2]]
[[1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]]
[[ 2.  0.  1. ... -0. -0. -0.]
 [ 0.  2.  0. ... -0. -0. -0.]
 [ 1.  0.  3. ... -0. -0. -0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


array([ 52.48427788,  47.51572212,  73.80490017,  40.58182779,
        35.61327204,  55.46138074,  50.49282499,  94.04579427,
        56.48247781,  23.25940543,  16.11713644,  18.29084967,
        11.14858068,  54.70154997,  97.97711287,  62.02288713,
        71.61153619,  38.38846381,  64.39220056,  28.43797482,
        27.16982462,  71.19613792,  37.97306554,  30.83079655,
        68.9831932 ,  33.02896747,  27.98783933,  77.52056407,
        72.47943593,  49.54723296,  13.59300723,  12.32485703,
         8.55187908,   7.28372888,  48.69929482,  42.88889679,
        37.84776864,  79.26333458,  68.50055037,  31.49944963,
        81.7175682 ,  44.71646747,  43.56596433,  73.22351536,
       116.77648464,  30.24385623,  23.10158724,  66.65455653,
        81.96084038,  78.03915962,  70.57525157,  69.42474843,
        90.6340751 ,  89.3659249 ,  53.57113449,  46.42886551,
        82.7163585 ,  45.60266112,  41.68098037,  78.55684869,
        41.44315131,  30.37357816,  29.10542795,  70.52

In [23]:
np.savetxt("E:\\A.csv", A, fmt='%d', delimiter=',')